<a href="https://colab.research.google.com/github/nguyenngoctruong2k1/NCKH_IDS/blob/main/IDS/KDDCup99/LSTM/lskeras2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import thư viện

In [34]:
from __future__ import print_function
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import LSTM, SimpleRNN, GRU
from keras.datasets import imdb
from keras.utils.np_utils import to_categorical
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import Normalizer
import h5py
from keras import callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

# Load data

In [36]:
from google.colab import drive
drive.mount('/content/drive')
traindata = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NCKH_IDS/kdd/kddtrain.csv', header=None)
testdata = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NCKH_IDS/kdd/kddtest.csv', header=None)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
protocol_type = {'icmp': 2, 'tcp': 0, 'udp': 1}
flag = {'OTH': 9,
        'REJ': 2,
        'RSTO': 6,
        'RSTOS0': 8,
        'RSTR': 7,
        'S0': 4,
        'S1': 1,
        'S2': 3,
        'S3': 5,
        'SF': 0,
        'SH': 10}

In [37]:
def F_label(data):
  if data == 'normal.':
    return 0
  return 1
def SoHoaDuLieu(traindata):
  traindata[1] = traindata[1].map(protocol_type)
  traindata[3] = traindata[3].map(flag)
  traindata[41] = traindata[41].map(F_label)
  # traindata[2] = traindata[2].map(service)
  return traindata.drop(2, axis = 1)

In [38]:
traindata = SoHoaDuLieu(traindata)
testdata = SoHoaDuLieu(testdata)

In [39]:
testdata

,0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41
0,0,1,0,105,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0,0
1,0,1,0,105,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0,0
2,0,1,0,105,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0,0
3,0,1,0,105,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0,1
4,0,1,0,105,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,254,1.0,0.01,0.01,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311024,0,1,0,105,147,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.00,0.01,0.0,0.0,0.0,0.0,0.0,0
311025,0,1,0,105,147,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.00,0.01,0.0,0.0,0.0,0.0,0.0,0
311026,0,1,0,105,147,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.00,0.01,0.0,0.0,0.0,0.0,0.0,0
311027,0,1,0,105,147,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.00,0.01,0.0,0.0,0.0,0.0,0.0,0


In [ ]:
X = traindata.iloc[:,0:40]
Y = traindata[41]
T = testdata.iloc[:,0:40]
C = testdata[41]

In [40]:
scaler = Normalizer().fit(X)
trainX = scaler.transform(X)
# summarize transformed data
np.set_printoptions(precision=3)
#print(trainX[0:5,:])

scaler = Normalizer().fit(T)
testT = scaler.transform(T)
# summarize transformed data
np.set_printoptions(precision=3)
#print(testT[0:5,:])


y_train = np.array(Y)
y_test = np.array(C)


# reshape input to be [samples, time steps, features]
X_train = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
X_test = np.reshape(testT, (testT.shape[0], 1, testT.shape[1]))


print(X_train.shape)

(494021, 1, 40)


In [41]:
y_train.shape

(494021,)

In [42]:
X_train.shape

(494021, 1, 40)

# Dựng model

In [68]:
batch_size = 32

# 1. define the network
model = Sequential()
model.add(LSTM(16,input_dim=40, return_sequences=True))  # try using a GRU instead, for fun
model.add(Dropout(0.1))
model.add(LSTM(16, return_sequences=True))  # try using a GRU instead, for fun
model.add(Dropout(0.1))
model.add(LSTM(16, return_sequences=False))  # try using a GRU instead, for fun
model.add(Dropout(0.1))
model.add(Dense(1))
model.add(Activation('sigmoid'))

print(model.get_config())

{'name': 'sequential_6', 'layers': [{'class_name': 'InputLayer', 'config': {'batch_input_shape': (None, None, 40), 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'lstm_8_input'}}, {'class_name': 'LSTM', 'config': {'name': 'lstm_8', 'trainable': True, 'batch_input_shape': (None, None, 40), 'dtype': 'float32', 'return_sequences': True, 'return_state': False, 'go_backwards': False, 'stateful': False, 'unroll': False, 'time_major': False, 'units': 16, 'activation': 'tanh', 'recurrent_activation': 'sigmoid', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'recurrent_initializer': {'class_name': 'Orthogonal', 'config': {'gain': 1.0, 'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'unit_forget_bias': True, 'kernel_regularizer': None, 'recurrent_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'recurrent_constraint': None, 'bias_constraint': None, '

# Train model

In [69]:
# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
checkpointer = callbacks.ModelCheckpoint(filepath="/content/drive/MyDrive/Colab Notebooks/NCKH_IDS/kdd/kddresults/lstmlayer/checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='val_acc',mode='max')
csv_logger = CSVLogger('/content/drive/MyDrive/Colab Notebooks/NCKH_IDS/kdd/kddresults/training_set_iranalysis.csv',separator=',', append=False)
model.fit(X_train, y_train, batch_size=batch_size, epochs=20, validation_data=(X_test, y_test),callbacks=[checkpointer,csv_logger])

Epoch 1/20
15439/15439 [==============================] - 95s 6ms/step - loss: 0.0773 - accuracy: 0.9728 - val_loss: 0.3939 - val_accuracy: 0.9155
Epoch 2/20
15439/15439 [==============================] - 87s 6ms/step - loss: 0.0192 - accuracy: 0.9938 - val_loss: 0.6332 - val_accuracy: 0.8174
Epoch 3/20
15439/15439 [==============================] - 87s 6ms/step - loss: 0.0147 - accuracy: 0.9952 - val_loss: 0.6486 - val_accuracy: 0.8161
Epoch 4/20
15439/15439 [==============================] - 90s 6ms/step - loss: 0.0125 - accuracy: 0.9954 - val_loss: 0.5638 - val_accuracy: 0.8370
Epoch 5/20
15439/15439 [==============================] - 88s 6ms/step - loss: 0.0111 - accuracy: 0.9960 - val_loss: 0.6030 - val_accuracy: 0.9097
Epoch 6/20
15439/15439 [==============================] - 89s 6ms/step - loss: 0.0111 - accuracy: 0.9960 - val_loss: 0.5423 - val_accuracy: 0.9201
Epoch 7/20
15439/15439 [==============================] - 90s 6ms/step - loss: 0.0101 - accuracy: 0.9965 - val_loss: 0

In [70]:
model.save("/content/drive/MyDrive/Colab Notebooks/NCKH_IDS/kdd/kddresults/lstm1layer/fullmodel/lstm1layer_model.hdf5")

# Report

In [61]:
loss, accuracy = model.evaluate(X_test, y_test)
print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy*100))

9720/9720 [==============================] - 14s 1ms/step - loss: 0.8460 - accuracy: 0.9214

Loss: 0.85, Accuracy: 92.14%


In [62]:
y_pred = model.predict_classes(X_test)
np.savetxt('/content/drive/MyDrive/Colab Notebooks/NCKH_IDS/kdd/kddresults/lstm1layer/lstm1predicted.txt', y_pred, fmt='%01d')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [63]:
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)

In [64]:
accuracy_score(y_test, y_pred)

0.9213706760462851

In [65]:
precision_score(y_test, y_pred, average="binary")

0.995474557541527

In [66]:
f1_score(y_test, y_pred, average="binary")

0.9488881457950176

In [67]:
recall_score(y_test, y_pred , average="binary")

0.9064671213403823